### 匯入套件

In [ ]:
import pandas as pd
import numpy as np
import os


### 匯入檔案

In [14]:
current_dir = os.getcwd()
input_path=os.path.join(current_dir,r"../../data/104_cleaned_0717.csv")

df=pd.read_csv(input_path)
print(df.head())

   20250714   1 多媒體傳播相關業 杰德創意影音管理股份有限公司  \
0  20250714   6  電腦軟體服務業       不晚科技有限公司   
1  20250715   5     停車場業   台灣國際開發事業有限公司   
2  20250715   5  廣告行銷公關業   華權數位整合行銷有限公司   
3  20250714   3  廣告行銷公關業     創浪行銷顧問有限公司   
4  20250714  10  網際網路相關業       盛形通運有限公司   

  【負責品牌】 全球LGBTQ+娛樂領導平台 GagaOOLala https://www.gagaoolala.com/en/home  【工作內容】 ●數據分析與報告： 1.管理影音串流OTT平台數據資料，透過資料分析和監測，確保平台營運維持良  \
0  1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...                                                                            
1  1. 串接各式資料來源，進行資料清洗、分析相關商業命題研究。 2. 深入了解業務需求，並將數...                                                                            
2  主要工作述敘： 1. 數據分析與精算： ■分析遊戲玩家行為數據、直播主表現數據，協助優化遊戲...                                                                            
3  數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。  2.消費者洞察: 掌握潛在目...                                                                            
4  一、工作內容? 業務數據需求分析：研究運送平台等同類型平臺模式，結合臺灣本地業務，梳理司機招...                                            

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 22 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [16]:
df.isnull().sum()

20250714                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               0
1                                                                                              

### 工作時段太多Null值，刪除該欄位

In [17]:
df.drop(columns=['工作時段','工作型態','職務分類代碼','薪資上限','福利制度'],inplace=True)

KeyError: "['工作時段', '工作型態', '職務分類代碼', '薪資上限', '福利制度'] not found in axis"

In [ ]:
df['薪資下限']=df['薪資下限'].replace(0,40000)    #將面議的改成法規規定最低40000
df=df[df['薪資下限']>=1000]     #將時薪(非正職、學生實習)的資料去除

In [ ]:
df.shape

(350, 15)

In [ ]:
df['major']

0      ['數理統計相關', '其他數學及電算機科學相關', '工業工程相關']
1            ['統計學相關', '資訊管理相關', '大眾傳播學相關']
2                  ['數學及電算機科學學科類', '工程學科類']
3            ['一般商業學類', '醫藥工程相關', '數理統計相關']
4                                        []
                       ...                 
356                     ['統計學相關', '數理統計相關']
357            ['經濟學相關', '會計學相關', '財稅金融相關']
358            ['資訊管理相關', '統計學相關', '經濟學相關']
359                ['數學及電算機科學學科類', '統計學相關']
360    ['資訊管理相關', '資訊工程相關', '其他數學及電算機科學相關']
Name: major, Length: 350, dtype: object

In [ ]:
import ast # 導入 ast 模組
df["是否要求特定major"] = df['major'].apply(lambda x: 1 if len(ast.literal_eval(x)) > 0 else 0)
df.drop(columns="major",inplace=True)

In [ ]:
df['是否要求特定major'].tail(20)

340    0
341    1
342    0
343    0
344    1
345    1
346    1
347    0
348    1
350    0
351    0
352    0
353    0
354    0
355    0
356    1
357    1
358    1
359    1
360    1
Name: 是否要求特定major, dtype: int64

In [ ]:
df['是否遠距'].value_counts()

是否遠距
0    320
2     27
1      3
Name: count, dtype: int64

In [ ]:
df['是否遠距'] = df['是否遠距'].astype(int)

In [ ]:
remote_work={0:"無遠距",1:"完全遠距",2:"混和"}
df['是否遠距']=df['是否遠距'].map(remote_work)

In [ ]:
df.head()

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major
0,20250709,10,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1
1,20250714,1,廣告行銷公關業,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,"[4, 5, 6]",無遠距,40000,0,1
2,20250708,6,電腦系統整合服務業,艾捷科技有限公司,我們正在尋找身懷絕技的【數據分析師】加入我們的團隊\n\n你的任務：\n\n 解讀數據...,台中市,數據分析師,14437328,24.148053,120.645606,"[4, 5]",無遠距,40000,1,1
3,20250625,9,工商顧問服務業,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"[2, 3, 4, 5, 6]",無遠距,50000,1,1
4,20250714,10,電腦軟體服務業,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,"[3, 4]",無遠距,40000,1,0


In [ ]:
df[df["工作編號"].duplicated()]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major
69,20250709,10,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1
269,20250708,18,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\n2. 大數據分析與建模。\n3. 舉辦內部統計與數值分析的教...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"[5, 6]",無遠距,40000,0,1


In [ ]:
df[df["工作編號"].isin([14720195,13634345])]

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major
0,20250709,10,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1
69,20250709,10,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1
268,20250708,18,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\n2. 大數據分析與建模。\n3. 舉辦內部統計與數值分析的教...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"[5, 6]",無遠距,40000,0,1
269,20250708,18,光電產業,達興材料股份有限公司,1. 執行AI技術開發專案。\n2. 大數據分析與建模。\n3. 舉辦內部統計與數值分析的教...,台中市,研發-資料科學研究員,13634345,24.204932,120.607608,"[5, 6]",無遠距,40000,0,1


In [ ]:
df.drop_duplicates(subset="工作編號",keep="first",inplace=True)  #保留第一個

In [ ]:
df.set_index("工作編號")

,上架日期,申請人數,產業,公司名稱,description,縣市,職稱,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major
工作編號,,,,,,,,,,,,,,
14720195,20250709,10,家用電器製造業,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,25.068090,121.455088,[4],無遠距,38000,0,1
14724608,20250714,1,廣告行銷公關業,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,25.034500,121.560053,"[4, 5, 6]",無遠距,40000,0,1
14437328,20250708,6,電腦系統整合服務業,艾捷科技有限公司,我們正在尋找身懷絕技的【數據分析師】加入我們的團隊\n\n你的任務：\n\n 解讀數據...,台中市,數據分析師,24.148053,120.645606,"[4, 5]",無遠距,40000,1,1
14079349,20250625,9,工商顧問服務業,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,25.053372,121.522852,"[2, 3, 4, 5, 6]",無遠距,50000,1,1
14325959,20250714,10,電腦軟體服務業,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,25.076077,121.575436,"[3, 4]",無遠距,40000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12828775,20250710,50,其它軟體及網路相關業,遊戲橘子數位科技股份有限公司,此為橘子集團旗下的行動支付服務品牌， 全面推廣電子支付的服務版圖。我們致力推廣電子支付應用範...,台北市,數據分析師,25.071724,121.576463,"[3, 4, 5]",無遠距,40000,1,1
14676852,20250710,9,其他半導體相關業,怡凡得股份有限公司,1. Lead the reconciliation and validation in f...,新北市,Global Finance Data Analyst,25.058571,121.458476,"[4, 5]",無遠距,40000,1,1
14516906,20250714,13,電腦系統整合服務業,陣列科技有限公司,主責參與資料分析專案、建置儀錶板，協助使用者從資料中獲取洞察結果，落實數據驅動決策，持續優化...,台北市,外商（FMCG）資料分析人員,25.040920,121.572006,"[3, 4, 5]",無遠距,40000,1,1


In [ ]:
from cleaning_industry import clean_industry_column
df=clean_industry_column(df)
df.drop(columns=["產業"],inplace=True)

In [ ]:
df["八大產業"].value_counts()

八大產業
資訊科技業       101
服務與專業顧問業     76
金融與保險業       62
製造業          50
批發與零售業       34
醫療與健康照護      12
觀光與餐飲業        6
運輸與物流業        5
其他            2
Name: count, dtype: int64

In [ ]:
df.head()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major,八大產業
0,20250709,10,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1,製造業
1,20250714,1,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,"[4, 5, 6]",無遠距,40000,0,1,服務與專業顧問業
2,20250708,6,艾捷科技有限公司,我們正在尋找身懷絕技的【數據分析師】加入我們的團隊\n\n你的任務：\n\n 解讀數據...,台中市,數據分析師,14437328,24.148053,120.645606,"[4, 5]",無遠距,40000,1,1,資訊科技業
3,20250625,9,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"[2, 3, 4, 5, 6]",無遠距,50000,1,1,服務與專業顧問業
4,20250714,10,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,"[3, 4]",無遠距,40000,1,0,資訊科技業


In [ ]:
df

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major,八大產業
0,20250709,10,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,[4],無遠距,38000,0,1,製造業
1,20250714,1,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,"[4, 5, 6]",無遠距,40000,0,1,服務與專業顧問業
2,20250708,6,艾捷科技有限公司,我們正在尋找身懷絕技的【數據分析師】加入我們的團隊\n\n你的任務：\n\n 解讀數據...,台中市,數據分析師,14437328,24.148053,120.645606,"[4, 5]",無遠距,40000,1,1,資訊科技業
3,20250625,9,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,"[2, 3, 4, 5, 6]",無遠距,50000,1,1,服務與專業顧問業
4,20250714,10,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,"[3, 4]",無遠距,40000,1,0,資訊科技業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,20250710,50,遊戲橘子數位科技股份有限公司,此為橘子集團旗下的行動支付服務品牌， 全面推廣電子支付的服務版圖。我們致力推廣電子支付應用範...,台北市,數據分析師,12828775,25.071724,121.576463,"[3, 4, 5]",無遠距,40000,1,1,資訊科技業
357,20250710,9,怡凡得股份有限公司,1. Lead the reconciliation and validation in f...,新北市,Global Finance Data Analyst,14676852,25.058571,121.458476,"[4, 5]",無遠距,40000,1,1,製造業
358,20250714,13,陣列科技有限公司,主責參與資料分析專案、建置儀錶板，協助使用者從資料中獲取洞察結果，落實數據驅動決策，持續優化...,台北市,外商（FMCG）資料分析人員,14516906,25.040920,121.572006,"[3, 4, 5]",無遠距,40000,1,1,資訊科技業
359,20250709,13,博晶醫電股份有限公司,工作職責：\n1. 數據分析與質量控制：\n ‧ 監控算法在不同訊號環境中的表現...,新北市,資料分析工程師,14098922,24.984257,121.540141,"[4, 5]",無遠距,40000,1,1,資訊科技業


In [ ]:
df["學歷要求"].value_counts()

學歷要求
[4, 5, 6]             117
[4, 5]                102
[3, 4, 5, 6]           34
[1, 2, 3, 4, 5, 6]     25
[3, 4, 5]              17
[4]                    14
[5]                    12
[5, 6]                  9
[3, 4]                  8
[2, 3, 4, 5, 6]         7
[2, 3, 4, 5]            1
[2, 3, 4]               1
[6]                     1
Name: count, dtype: int64

In [ ]:
edu_map = {
            1: '不拘',
            2: '高中',
            3: '專科',
            4: '大學',
            5: '碩士',
            6: '博士',
        }
df['學歷要求']=df['學歷要求'].apply(lambda x:edu_map.get(min(ast.literal_eval(x))))

In [ ]:
df.head()

,上架日期,申請人數,公司名稱,description,縣市,職稱,工作編號,經度,緯度,學歷要求,是否遠距,薪資下限,是否積極徵才,是否要求特定major,八大產業
0,20250709,10,金筆企業股份有限公司,1. 工廠生產數據收集、整理與分析，協助相關人員了解生產狀況\n2. 建立分析指標，追蹤並記...,新北市,數據分析工程師,14720195,25.068090,121.455088,大學,無遠距,38000,0,1,製造業
1,20250714,1,創浪行銷顧問有限公司,數據分析工具監測社群媒體、論壇、新聞平台，定期監控社群輿情。\n\n2.消費者洞察: 掌握潛...,台北市,數據分析經理,14724608,25.034500,121.560053,大學,無遠距,40000,0,1,服務與專業顧問業
2,20250708,6,艾捷科技有限公司,我們正在尋找身懷絕技的【數據分析師】加入我們的團隊\n\n你的任務：\n\n 解讀數據...,台中市,數據分析師,14437328,24.148053,120.645606,大學,無遠距,40000,1,1,資訊科技業
3,20250625,9,世銳顧問有限公司,題，為客戶提供具有價值的服務並且協助推動客戶的財務成長。服務內容包括：\n‧\t打造具有吸引...,台北市,商業數據分析師,14079349,25.053372,121.522852,高中,無遠距,50000,1,1,服務與專業顧問業
4,20250714,10,不晚科技有限公司,1.熟練數據分析工具，能進行數據清理、分析和建模，分析市場營銷活動的效果，提供報告和改進建議...,台北市,數據分析,14325959,25.076077,121.575436,專科,無遠距,40000,1,0,資訊科技業


In [ ]:
#暫時匯出
#desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "暫時匯出查看.csv")
#df.to_csv(desktop_path, index=False, encoding='utf-8-sig')